In [1]:
import pandas as pd
import numpy as np
import os
import re
import math
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import matplotlib
import matplotlib.gridspec as gridspec
import scipy.signal as signal
from sklearn import preprocessing

sns.set(color_codes=True)

pd.set_option('display.height', 300)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# data = a numpy array containing the signal to be processed
# fs = a scalar which is the sampling frequency of the data
chunkdf=pd.read_csv('/home/wenlu/Documents/XPFIFA/biopacMatch/S49_m21.txt',sep='\t',header=None,usecols=[0,1,3,6,7,9],names=['ts','rsp', 'emg_s','eda','ecg','emg_f'])
data = chunkdf['eda'].as_matrix()  
data

height has been deprecated.



array([ 18.0405,  18.0374,  18.0405, ...,  19.8425,  19.8441,  19.8441])

In [178]:
data=preprocessing.scale(data)
data

array([ 1.03056101,  1.02838908,  1.03056101, ...,  2.29308162,
        2.29420261,  2.29420261])

In [179]:
plt.plot(data)
plt.show()

In [130]:
data[::1000].size

241

In [180]:
overlap_fac=0.5
fft_size =2000
fs=1000
hop_size = np.int32(np.floor(fft_size * (1-overlap_fac)))
pad_end_size = fft_size          # the last segment can overlap the end of the data array by no more than one window size
total_segments = np.int32(np.ceil(len(data) / np.float32(hop_size)))
t_max = len(data) / np.float32(fs)
 
window = np.hanning(fft_size)  # our half cosine window
inner_pad = np.zeros(fft_size) # the zeros which will be used to double each segment size
 
proc = np.concatenate((data, np.zeros(pad_end_size)))              # the data to process
result = np.empty((total_segments, fft_size), dtype=np.float32)    # space to hold the result
 
for i in xrange(total_segments):                      # for each segment
    current_hop = hop_size * i                        # figure out the current segment offset
    segment = proc[current_hop:current_hop+fft_size]  # get the current segment
    windowed = segment * window                       # multiply by the half cosine function
    padded = np.append(windowed, inner_pad)           # add 0s to double the length of the data
    spectrum = np.fft.fft(padded) / fft_size          # take the Fourier Transform and scale by the number of samples
    autopower = np.abs(spectrum * np.conj(spectrum))  # find the autopower spectrum
    result[i, :] = autopower[:fft_size]               # append to the results array

In [181]:
result = 20*np.log10(result)          # scale to db

In [182]:
#tmp=result[:,0:200]
tmp=result[:]

In [194]:
result

array([[ -11.8964138 ,  -14.73466015,  -23.89496803, ..., -224.9045105 ,
        -195.04241943, -185.33706665],
       [ -12.58586025,  -15.42796993,  -24.60222626, ..., -187.89422607,
        -190.16340637, -198.32110596],
       [ -13.33389854,  -16.18060303,  -25.37310791, ..., -216.01875305,
        -191.89553833, -185.6761322 ],
       ..., 
       [   1.68175745,   -1.07095122,   -9.89243889, ..., -197.5173645 ,
        -206.16676331, -217.91854858],
       [  -9.33522415,  -10.24275589,  -13.00170708, ..., -129.5533905 ,
        -129.53747559, -129.53767395],
       [         -inf,          -inf,          -inf, ...,          -inf,
                 -inf,          -inf]], dtype=float32)

In [190]:
plt.hist(tmp.ravel(), bins=256, range=(-250, -100), fc='k', ec='k')
plt.show()

In [195]:
tmp.shape

(200, 241)

In [191]:
img = plt.imshow(tmp, cmap='spectral',aspect='auto',clim=(-250,-150))
plt.colorbar()
plt.show()

In [193]:
chunkdf=pd.read_csv('/home/wenlu/Documents/XPFIFA/biopacMatch/S49_m21.txt',sep='\t',header=None,usecols=[0,1,3,6,7,9],names=['ts','rsp', 'emg_s','eda','ecg','emg_f'])
data = chunkdf['eda'].as_matrix()  
data=preprocessing.scale(data)

dataseq=data[::1000]
l=dataseq.size

overlap_fac=0.5
fft_size =2000
fs=1000
hop_size = np.int32(np.floor(fft_size * (1-overlap_fac)))
pad_end_size = fft_size          # the last segment can overlap the end of the data array by no more than one window size
total_segments = np.int32(np.ceil(len(data) / np.float32(hop_size)))
t_max = len(data) / np.float32(fs)
 
window = np.hanning(fft_size)  # our half cosine window
inner_pad = np.zeros(fft_size) # the zeros which will be used to double each segment size
 
proc = np.concatenate((data, np.zeros(pad_end_size)))              # the data to process
result = np.empty((total_segments, fft_size), dtype=np.float32)    # space to hold the result
 
for i in xrange(total_segments):                      # for each segment
    current_hop = hop_size * i                        # figure out the current segment offset
    segment = proc[current_hop:current_hop+fft_size]  # get the current segment
    windowed = segment * window                       # multiply by the half cosine function
    padded = np.append(windowed, inner_pad)           # add 0s to double the length of the data
    spectrum = np.fft.fft(padded) / fft_size          # take the Fourier Transform and scale by the number of samples
    autopower = np.abs(spectrum * np.conj(spectrum))  # find the autopower spectrum
    result[i, :] = autopower[:fft_size]               # append to the results array
result = 20*np.log10(result) 
tmp=result[:,0:200]
tmp=tmp.transpose()[::-1]

fig = plt.figure(figsize=(40,30))
gs = gridspec.GridSpec(7,1)
 # raw signal

ax1 = fig.add_subplot(gs[1:2, 0])
ax1.plot(range(l), dataseq, linewidth=1, label='signal')
#ax1.legend()
ax1.set_ylabel('signal')


ax2 = fig.add_subplot(gs[3:7, 0], sharex=ax1)
ax2.imshow(tmp, cmap='spectral',clim=(-200,-100))
#ax2.legend()
ax2.set_ylabel('stft')


#savename='./../plot/'+figurename+'.pdf'
#fig.savefig(savename,dpi=100)
plt.show()

In [ ]:
#result = 20*np.log10(result)          # scale to db
#result = np.clip(result, -40, 200)    # clip values